In [2]:
import pandas as pd
import DB as db
import numpy as np

In [8]:
log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2022-12-11.zip"
log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-20.zip"
# log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-19.zip"
# log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-18.zip"

logdf= pd.read_csv(log_summary_file_name, low_memory=False).query("token == 'confirmLandReservation True'")\
    .drop(columns=['node', 'task_id','project_id', 'error_line'])
# logdf.head()

In [9]:
# logdf.query("Land_ID in (80312, 80487)")
logdf.query("NID == '26312132702012'")

,log_date,line_no,NID,log_type,country,IP_address,service,token,categ,Gov,Land_ID,City,Region,District,Sub_District,Land_No,land_size,excellence_ratio,checksum,NA1
69096,2023-04-20 10:16:43,3834604,26312132702012,ERROR,AE,217.165.114.211,confirmLandReservation,confirmLandReservation True,user,قنا,80487.0,قنا الجديدة,منطقة 600 فدان التوسعات الشرقية,-,الثانية,498.0,209.0,1.06,015975F7E08B1CA5FB85BE63226E492A48BDF0168BC5AE...,NaN
69098,2023-04-20 10:16:43,3834626,26312132702012,ERROR,EG,196.133.30.103,confirmLandReservation,confirmLandReservation True,user,قنا,80312.0,قنا الجديدة,منطقة 600 فدان التوسعات الشرقية,-,الثانية,241.0,209.0,1.06,017B780E3DC6A2E24581C9763DE7CE9BEE894EEC745246...,NaN


In [14]:
land_app_fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\DB-summary\DB checksum\2023-05-03\NewQueryLand.zip"
dbdf= pd.read_csv(land_app_fn, low_memory=False, dtype={'applicant_national_id':str}).query("project_id == 86")
    # .query("project_id == 86 and land_application_status == 'W'")[['applicant_national_id', 'check_sum', 'land_id', 'project_id']]

In [15]:
dbdf.query("land_id == '80487'")

,project_id,applicant_national_id,check_sum,land_id


In [16]:
df = (dbdf.merge(logdf, left_on='applicant_national_id', right_on='NID')
      .assign(match = lambda x:(x.checksum == x.check_sum))
    #   .fillna('land_id', axis=1)
    #   .assign(land_id = lambda x: x.astype(np.int64) )
    )
# df.info()

In [17]:
df.match.value_counts()

match
True     999
False      1
Name: count, dtype: int64

In [20]:
df.query("match == False").assign(land_id = lambda x: x.land_id.astype(np.int64)).to_excel(r"C:\Users\yahia\Downloads\mismatch.xlsx")

## UNITS

In [28]:
log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-02-07.zip"

logdf= pd.read_csv(log_summary_file_name, low_memory=False, dtype={'NID':str}).query("token == 'confirmReservation True'")\
    .drop(columns=['node', 'task_id','project_id', 'error_line'])



In [35]:
logdf.query("NID == '28306040103632'")

,log_date,line_no,NID,log_type,country,IP_address,service,token,categ,Gov,City,Region,District,Sub_District,Floor_No,building_no,Unit_No,Unit_Model,Unit_ID,checksum
37004,2023-02-07 10:04:19,5510745,28306040103632,ERROR,EG,196.153.52.114,confirmReservation,confirmReservation True,user,القاهرة,القاهرة الجديدة,نزهة الأندلس عمارات مدخل (أ),NaN,NaN,0.0,9.0,15.0,-,157579.0,0116ECD030C0448E25A2CD6851EAF9D4915158DCE20E79...


In [38]:
unit_app_fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\DB-summary\DB checksum\2023-05-03\NewQueryUnit.zip"
dbdf= pd.read_csv(unit_app_fn, low_memory=False, dtype={'applicant_national_id':str}).query("project_id == 82")

In [39]:
dbdf.query("applicant_national_id == '28306040103632'")

,project_id,applicant_national_id,check_sum,unit_id
6745,82,28306040103632,015E7F7F4A31AD630BCBDC636635A97470BB0CA428C44A...,155895


In [40]:
df = (dbdf.merge(logdf, left_on='applicant_national_id', right_on='NID')
      .assign(match = lambda x:(x.checksum == x.check_sum))
)


In [41]:
df.match.value_counts()

match
False    1
Name: count, dtype: int64

In [42]:
df.query("match == False").assign(unit_id = lambda x: x.unit_id.astype(np.int64))[['NID', 'check_sum', 'checksum']]
#.to_excel(r"C:\Users\yahia\Downloads\mismatch.xlsx")

,NID,check_sum,checksum
0,28306040103632,015E7F7F4A31AD630BCBDC636635A97470BB0CA428C44A...,0116ECD030C0448E25A2CD6851EAF9D4915158DCE20E79...
